In [2]:
import copy
import os
import pickle
import random
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.backends.cudnn
from sacred import Experiment
from sacred.commands import print_config, save_config
from sacred.observers import FileStorageObserver
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from fail_observer import FailObserver
from model import *
from model.constants import *
from model.dataset import (OriginalMAPS, SynthesizedInstruments,
                           SynthesizedTrumpet, CustomBatchDataset)
from model.evaluate_fn import evaluate_wo_velocity
from snapshot import Snapshot

/home/users/twende/miniconda3/envs/music-transcription-frozen/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/users/twende/miniconda3/envs/music-transcription-frozen/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
# load model

model = UnetTranscriptionModel(
    ds_ksize=(2, 2), ds_stride=(2, 2), mode="imagewise",
    spec='CQT', norm=1, device="cuda:1", logdir="unet_temp", debug_mode=False
)
model.to("cuda:1")
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10000, gamma=0.98)

# load model, optimizer, scheduler, batch, epoch and batch_idx from test.pt
snapshot = torch.load("error_snapshot_ep_batch.pt")

model.load_my_state_dict(snapshot['model_state_dict'])
optimizer.load_state_dict(snapshot['optimizer_state_dict'])
scheduler.load_state_dict(snapshot['scheduler_state_dict'])

batch = snapshot['batch']
ep = snapshot['epoch']
batch_idx = snapshot['batch_idx']

CQT kernels created, time used = 0.1790 seconds


/tmp/ipykernel_1680394/4241064724.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  snapshot = torch.load("results/unet_model_trained_on_GuitarSet/model-50.pt")


KeyError: 'model_state_dict'

In [ ]:
for key in batch.keys():
    if key == "path": continue
    
    batch[key] = batch[key].to("cuda:1")

In [5]:
predictions, losses, _ = model.run_on_batch(batch, batch_description=str(ep), batch_identifier=batch_idx)

In [9]:
total_loss = 0
loss = sum(losses.values())
total_loss += loss.item()
loss.backward()